# импорты

In [2]:
import pandas as pd
import boto3

In [3]:
import sys
print(sys.path)

['/Applications/PyCharm.app/Contents/plugins/python/helpers-pro/jupyter_debug', '/Applications/PyCharm.app/Contents/plugins/python/helpers/pydev', '/Users/dariamishina/Documents/HSE/YearProject/mlds23-authorship-identification/data_preproc/notebooks', '/Users/dariamishina/Documents/HSE/YearProject', '/Users/dariamishina/.conda/envs/untitled/lib/python38.zip', '/Users/dariamishina/.conda/envs/untitled/lib/python3.8', '/Users/dariamishina/.conda/envs/untitled/lib/python3.8/lib-dynload', '', '/Users/dariamishina/.conda/envs/untitled/lib/python3.8/site-packages']


In [4]:
# добавляем в path папку с тем файлом, откуда надо импортировать функции
sys.path.append('/Users/dariamishina/Documents/HSE/YearProject/mlds23-authorship-identification')

In [5]:
from data_preproc.src.utils import text_splitter

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dariamishina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/dariamishina/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
# sys.path.remove('/Users/dariamishina/Documents/HSE/YearProject/mlds23-authorship-identification')

# загружаем тексты из s3

In [ ]:
BUCKET_NAME = 'mlds23-authorship-identification'

In [ ]:
session = boto3.session.Session()

In [ ]:
s3 = session.client(
    service_name='s3',
    endpoint_url='https://storage.yandexcloud.net',
    aws_access_key_id='YCAJE6bQjBm5o7PaMjYqDKLFT',
    aws_secret_access_key='YCOVpYeJO8yux1IROpFDoGLkXxKZAIIrAdU0bZZ6',
    region_name='ru-cental1'
)

In [ ]:
for key in s3.list_objects(Bucket=BUCKET_NAME)['Contents']:
    print(key['Key'])

# загружаем и сразу же сплиттим на токены

Идея от Лены как разбивать произведения

1) Токенизировать nltk/spacy
2) Идем по списку токенов, как только набрали например 550 (это кстати точно гораздо больше, чем 5 предложений) — отрезаем все, которые были после последней запятой/точки/т.д. Т.е. если встреченный токен — пунктуация, то храним его индекс, потом срез до него

In [ ]:
total_df = []
for key in s3.list_objects(Bucket=BUCKET_NAME)['Contents']:
    try:
        if 'txt' in key['Key']: #фильтруем по расширению файла, нужны только txt
            file = s3.get_object(Bucket=BUCKET_NAME,Key=key['Key'])['Body']
            text = file.read().decode("utf-8")
            df = text_splitter(text) #вызываем функцию из utils для сплитования
            df['book'] = key['Key'] #добавляем название произведения в отдельный столбец
            total_df.append(df)
    except:
        print('не тот файл', key['Key'])
        continue

In [ ]:
total_df = pd.concat(total_df)
total_df.reset_index(drop=True, inplace=True)
total_df.to_csv('splitted_df.csv', index=False)

In [ ]:
DATA_PATH = '/home/d-mishina/texts/' #это локальный путь! Куда сохранился мой csv файл
DF_FILE_NAME = 'splitted_df.csv'

# загружаем в s3 

In [ ]:
s3.upload_file(f'{DATA_PATH}{DF_FILE_NAME}', BUCKET_NAME, f'splitted_data/{DF_FILE_NAME}')